In [1]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
from statistics import mean
from spellchecker import SpellChecker
spell = SpellChecker()

C:\Users\chris\AppData\Local\Programs\Python\Python310\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
df = pd.read_csv("C:\\Users\\chris\\Documents\\Job Documents\\Portfolio Evidence\\Research\\Health AI Tracking\\data_raw.csv", encoding="latin1")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8573 entries, 0 to 8572
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Column1               8573 non-null   int64  
 1   pmid                  8573 non-null   int64  
 2   abstract              8573 non-null   object 
 3   methods               8570 non-null   object 
 4   abstract_db           8573 non-null   object 
 5   abstract_db_score     8573 non-null   float64
 6   abstract_org          8573 non-null   object 
 7   abstract_org_score    8573 non-null   float64
 8   method_db             8573 non-null   object 
 9   method_db_score       8573 non-null   float64
 10  method_org            8573 non-null   object 
 11  method_org_score      8573 non-null   float64
 12  data_name             1444 non-null   object 
 13  data_source_location  1343 non-null   object 
 14  data_source_country   1352 non-null   object 
 15  open_access          

In [3]:
df.head(5)

,Column1,pmid,abstract,methods,abstract_db,abstract_db_score,abstract_org,abstract_org_score,method_db,method_db_score,...,method_org_score,data_name,data_source_location,data_source_country,open_access,region_db,local_db,study,collected,other
0,17168,31703390,"In emergency departments, the most common caus...",2.1. Study Design We report herein a retrosp...,quick sepsis-related organ failure assessment ...,0.071,hospital emergency department,0.071,chang gung research database,0.961,...,0.370,Chang Gung Research Database,Kaohsiung Chang Cheng Memorial Hospital,Taiwan,False,False,True,False,False,NaN
1,1743,31888592,Fetal heart rate (FHR) monitoring is a screeni...,Database description The data use...,test open-access database,0.089,test open-access database,0.120,ctu-uhb,0.932,...,0.568,Czech Technical University (CTU) Prague and th...,University Hospital in Brno,Czech Republic,True,False,True,False,False,NaN
2,6930,33882838,Predicting difficult airway is challengeable i...,This study was conducted after approval by th...,preanesthesia and anesthesia data of 1677 pati...,0.116,a single center,0.071,preanesthesia and anesthesia records,0.925,...,0.511,0,Chuncheon Sacred Heart Hospital,South Korea,False,False,False,False,True,NaN
3,2293,31505319,Methods of suicide have received considerable ...,In preparation for this study a literature re...,austria,0.274,Austria,0.429,statistics austria,0.924,...,0.666,Statistics Austria,0,Austria,True,True,False,False,False,NaN
4,20554,33332440,We present the results from a white-box machin...,Dataset The dataset used for our stu...,c5.0,0.349,C5.0,0.117,chapman university and shaoxing peopleÃ¢â¬â¢...,0.919,...,0.948,0,chapman university and shaoxing people's hospital,China,False,False,True,False,False,NaN


In [4]:
df_low = df.applymap(lambda s: s.lower() if type(s) == str else s)
df_low = pd.DataFrame(df_low.method_db.value_counts())
duplicates = df_low.loc[df_low.method_db >1]
duplicates.to_csv("C:\\Users\\chris\\Documents\\Job Documents\\Portfolio Evidence\\Research\\Health AI Tracking\\duplicates.csv")
duplicates.head(5)

,method_db
eeg,64
htis,52
tcga,43
ehr,38
medical records,37


In [5]:
duplicate_list = duplicates.index.values.tolist()
print(duplicate_list[0:5])

['eeg', 'htis', 'tcga', 'ehr', 'medical records']


In [6]:
df_new = df

#create new score
df_new["c_score"] = df_new["method_db_score"]

#+1 score if on duplicate list
df_new["c_score"] = df_new["c_score"] + df_new["method_db"].apply(lambda x: 1 if x in duplicate_list else 0)

#+1 score if brackets
df_new["c_score"] = df_new["c_score"] + df_new["method_db"].apply(lambda x: 1 if "(" in x else 0)

#+1 score if ends with database/dataset/databank/repository/cohort/study
df_new["c_score"] = df_new["c_score"] + df_new["method_db"].apply(lambda x: 2 if x.endswith(("database", "dataset", "databank", "repository", "cohort", "study", "survey", "project", "warehouse", "trial", "databases", "datasets", "databanks", "repositories", "cohorts", "studies", "surveys", "trials")) else 0)

#+1 score if non-english word (acronym) in text
def spell_check(word):
    #removes punctuation from str, returns T if single word is not in lang-ENG dict
    punct = """!"£$%^&*()_-+={[}]#:;@'<,>.?/"""
    nopunct = ""
    for char in word:
        if char not in punct:
            nopunct = nopunct + char
    split = nopunct.split(" ")
    for split_component in split:
        if split_component == "":
            split.remove("")
    return spell.unknown(split) == set()

df_new["c_score"] = df_new["c_score"] + df_new["method_db"].apply(lambda x: 0 if spell_check(x) else 1)

#reduce score to -1 if fuzzy matched health records & resets "method_db_score" added to c_score for exact matches to the mean for that match
def p_record_check(cell):
    #returns T if database is patient/medical records
    check_words = ["electronic health records", "patient records", "medical records", "electronic medical records", "electronic patient records", "electronic health record", "emr", "epr", "ehr", "ehrs", "electronic medical record system", "electronic health record system", "hospital information system", "hospital electronic medical record", "hospital electronic patient record", "a database", "the database", "picture-archiving and communication system", "PACS", "experimental dataset", "cancer dataset"]
    for check_word in check_words:
        if fuzz.ratio(str(cell), check_word) > 85:
            return True
    return False

df_new["c_score"] = df_new["method_db"].apply(lambda x: -1 if p_record_check(x) == True else mean(list(df_new.loc[df_new["method_db"] == x]["c_score"])))

#reduce score to -2 if fuzzy matched study investigation related terms
def ix_check(cell):
    #returns T if database is an investigation related term
    ix = ["mri", "ct", "x-ray", "eeg", "emg", "fmri", "ecg", "x-ray image", "ct image", "mri image", "DICOM", "PACS", "imagenet", "ultrasound", "ultrasound images"]
    for ix_word in ix:
        if fuzz.ratio(str(cell), ix_word) > 85:
            return True
    return False

df_new["c_score"] = df_new["method_db"].apply(lambda x: -2 if ix_check(x) == True else mean(list(df_new.loc[df_new["method_db"] == x]["c_score"])))

#reduce score to -3 if matched to other indicators of incorrect database name
df_new["c_score"] = df_new["method_db"].apply(lambda x: -3 if x.startswith(("department", "college", "institute", "university", "institutional", "our", "patient", "retrospective", "prospective", "training", "eth", "hte", "htis")) == True else mean(list(df_new.loc[df_new["method_db"] == x]["c_score"])))

#reduce score to -4 if picks up institute/hospital/university name
df_new["c_score"] = df_new["method_db"].apply(lambda x: -4 if x.endswith(("hospital", "institute", "university")) == True else mean(list(df_new.loc[df_new["method_db"] == x]["c_score"])))

#reduce score to -5 if db starts with http
df_new["c_score"] = df_new["method_db"].apply(lambda x: -5 if x.startswith("http") else mean(list(df_new.loc[df_new["method_db"] == x]["c_score"])))

df_new["c_score"].head(5)

0    3.550667
1    1.932000
2    1.925000
3    0.924000
4   -4.000000
Name: c_score, dtype: float64

In [7]:
#as above but for abstract_db
df_new["c_score2"] = df_new["abstract_db_score"]
df_new["c_score2"] = df_new["c_score2"] + df_new["abstract_db"].apply(lambda x: 1 if x in duplicate_list else 0)
df_new["c_score2"] = df_new["c_score2"] + df_new["abstract_db"].apply(lambda x: 1 if "(" in x else 0)
df_new["c_score2"] = df_new["c_score2"] + df_new["abstract_db"].apply(lambda x: 2 if x.endswith(("database", "dataset", "databank", "repository", "cohort", "study", "survey", "project", "warehouse", "trial", "databases", "datasets", "databanks", "repositories", "cohorts", "studies", "surveys", "trials")) else 0)
df_new["c_score2"] = df_new["c_score2"] + df_new["abstract_db"].apply(lambda x: 0 if spell_check(x) else 1)
df_new["c_score2"] = df_new["abstract_db"].apply(lambda x: -1 if p_record_check(x) == True else mean(list(df_new.loc[df_new["abstract_db"] == x]["c_score2"])))
df_new["c_score2"] = df_new["abstract_db"].apply(lambda x: -2 if ix_check(x) == True else mean(list(df_new.loc[df_new["abstract_db"] == x]["c_score2"])))
df_new["c_score2"] = df_new["abstract_db"].apply(lambda x: -3 if x.startswith(("department", "college", "institute", "university", "institutional", "our", "patient", "retrospective", "prospective", "training", "eth", "hte", "htis")) == True else mean(list(df_new.loc[df_new["abstract_db"] == x]["c_score2"])))
df_new["c_score2"] = df_new["abstract_db"].apply(lambda x: -4 if x.endswith(("hospital", "institute", "university")) == True else mean(list(df_new.loc[df_new["abstract_db"] == x]["c_score2"])))
df_new["c_score2"] = df_new["abstract_db"].apply(lambda x: -5 if x.startswith("http") else mean(list(df_new.loc[df_new["abstract_db"] == x]["c_score2"])))

df_new["c_score2"].head(5)

0    2.0710
1    3.0890
2    1.1160
3    0.2740
4    1.1865
Name: c_score2, dtype: float64

In [8]:
#new column for highest scoring db
df_new["best_score"] = df_new["method_db"].apply(lambda x: mean(df_new.loc[df_new["method_db"] == x]["c_score"]) if mean(df_new.loc[df_new["method_db"] == x]["c_score"]) > mean(df_new.loc[df_new["method_db"] == x]["c_score2"]) else mean(df_new.loc[df_new["method_db"] == x]["c_score2"]))
df_new["best_score"].head(5)

0    3.550667
1    3.089000
2    1.925000
3    0.924000
4    1.186500
Name: best_score, dtype: float64

In [9]:
#create column for db with highest score
df_new["best_match"] = df_new["method_db"].apply(lambda x: x if mean(df_new.loc[df_new["method_db"] == x]["c_score"]) > mean(df_new.loc[df_new["method_db"] == x]["c_score2"])  else max(df_new.loc[df_new["method_db"] == x]["abstract_db"]))
df_new["best_match"].head(5)

0            chang gung research database
1               test open-access database
2    preanesthesia and anesthesia records
3                      statistics austria
4                                    c5.0
Name: best_match, dtype: object

In [10]:
#create file with both c_scores and sort
#df_new = df_new.sort_values(by=["best_score", "best_match"],ascending=False)
df_new.to_csv("C:\\Users\\chris\\Documents\\Job Documents\\Portfolio Evidence\\Research\\Health AI Tracking\\c_score.csv")
df_new.head(20)

,Column1,pmid,abstract,methods,abstract_db,abstract_db_score,abstract_org,abstract_org_score,method_db,method_db_score,...,open_access,region_db,local_db,study,collected,other,c_score,c_score2,best_score,best_match
0,17168,31703390,"In emergency departments, the most common caus...",2.1. Study Design We report herein a retrosp...,quick sepsis-related organ failure assessment ...,0.071,hospital emergency department,0.071,chang gung research database,0.961,...,False,False,True,False,False,NaN,3.550667,2.071000,3.550667,chang gung research database
1,1743,31888592,Fetal heart rate (FHR) monitoring is a screeni...,Database description The data use...,test open-access database,0.089,test open-access database,0.120,ctu-uhb,0.932,...,True,False,True,False,False,NaN,1.932000,3.089000,3.089000,test open-access database
2,6930,33882838,Predicting difficult airway is challengeable i...,This study was conducted after approval by th...,preanesthesia and anesthesia data of 1677 pati...,0.116,a single center,0.071,preanesthesia and anesthesia records,0.925,...,False,False,False,False,True,NaN,1.925000,1.116000,1.925000,preanesthesia and anesthesia records
3,2293,31505319,Methods of suicide have received considerable ...,In preparation for this study a literature re...,austria,0.274,Austria,0.429,statistics austria,0.924,...,True,True,False,False,False,NaN,0.924000,0.274000,0.924000,statistics austria
4,20554,33332440,We present the results from a white-box machin...,Dataset The dataset used for our stu...,c5.0,0.349,C5.0,0.117,chapman university and shaoxing peopleÃ¢â¬â¢...,0.919,...,False,False,True,False,False,NaN,-4.000000,1.186500,1.186500,c5.0
5,251,33746600,"Anemia, for which erythropoiesis-stimulating a...",Patients and datasets Ethics statement Clin...,administration direction data,0.027,medical insurance systems. An artificial-intel...,0.050,electronic health records,0.916,...,False,False,False,False,True,NaN,-1.000000,0.027000,0.056026,university of massachusetts' eicu
6,5031,34709180,"Hepatocellular carcinoma (HCC), usually known ...",Data Sources We collected data from Taiwanes...,national health insurance research database,0.604,National Health Insurance Research Database of...,0.405,taiwanese national health insurance research d...,0.916,...,False,True,False,False,False,NaN,2.916000,3.641222,3.641222,national health insurance research database
7,4022,30463545,Osteoarthritis (OA) is the most common disease...,Data The data we used come from OAI which is...,public knee oa studies,0.229,public knee OA studies,0.398,oai,0.911,...,True,True,False,False,False,NaN,2.240000,2.229000,2.240000,oai
8,25130,34797894,Overdose prescription errors sometimes cause s...,This study was approved by the Ethics Committ...,uptodate database,0.077,Kyushu University Hospital,0.687,ehrs,0.907,...,False,False,False,False,True,NaN,-1.000000,3.077000,0.993700,uptodate database
9,1721,33623450,Considering the current situation of the novel...,Study Design and Participants Thi...,wnh,0.391,Zhongnan Hospital of Wuhan University,0.393,electronic medical records,0.903,...,False,False,False,False,True,NaN,-1.000000,1.391000,0.042088,wuhan
